# Введение в машинное обучение для Java-разработчиков
### Практическое задание 2.  Деревья решений, случайный лес.
### Дата выдачи: 26.10.2023

### Дедлайн: 23:59MSK 08.11.2023

## О задании
В этом задании мы попытаемся разобраться в устройстве деревьев решений и ансамбля на основе деревьев. 

## Оценивание и штрафы
Каждая из задач (помечены тегом [task]) имеет определенное количество баллов (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. 

- от 4 до 8 баллов - оценка "3"
- от 8 до 14 баллов - оценка "4"
- 15 баллов - оценка "5"

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов, что автоматически ведет к несдаче курса. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в комментариях. 
В данном задании есть необязательные бонусные задания, выполнение которых добавляет баллы в карму :)

## Формат сдачи
Задания сдаются путем форка основного репозитория, коммита решения в мастер-ветку вашего форка и оповещении преподавателя о выполнении ДЗ. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

[[Укажите количество набранных баллов]]

## Часть 1. Дерево решений

- [task] Реализуйте подбор признака и порога для поиска условия разбиения выборки методом перебора (2 балла)
- [task] Реализуйте метод predict для инференса дерева на датасете (1 балл)
- [task] Обучите дерево на датасете ирисов Фишера (1 балл)
- Бонусное задание: В методе `_calculate_leaf_value` возвращайте вероятность классов
- Бонусное задание: Замените критерий Джини на энтропийный критерий
4 балла

In [87]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

class DecisionNode:
    def __init__(self, feature=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split:
            return DecisionNode(value=self._calculate_leaf_value(y))

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None or best_threshold is None:
            return DecisionNode(value=self._calculate_leaf_value(y))

        mask = X[:, best_feature] <= best_threshold
        true_branch = self._build_tree(X[mask], y[mask], depth + 1)
        false_branch = self._build_tree(X[~mask], y[~mask], depth + 1)

        return DecisionNode(feature=best_feature, threshold=best_threshold, true_branch=true_branch, false_branch=false_branch)

    def _find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]
        
        for i in range(n_features):
            curXColumn = X[:, i]
            cur_gain, cur_threshold =  self._find_best_split_on_column(curXColumn, y)

            if cur_gain > best_gain:
                best_gain = cur_gain
                best_feature = i
                best_threshold = cur_threshold
            
        return best_feature, best_threshold

    def _split_y(self, x_column, y_values, x_split_value):
        y_true = [y for idx, element in enumerate(x_column) if element <= x_split_value for y in [y_values[idx]]]
        y_false = [y for idx, element in enumerate(x_column) if element > x_split_value for y in [y_values[idx]]]
        return y_true, y_false

    def _find_best_split_on_column(self, x_column, y_values):
        column_best_gain = 0
        column_best_threshold = None
        for idx, element in enumerate(x_column):
            y_true, y_false = self._split_y(x_column, y_values, element)
            cur_gain = self._calculate_gain(y_values, y_true ,y_false)
            if column_best_gain < cur_gain:
                column_best_gain = cur_gain
                column_best_threshold = element
        return column_best_gain, column_best_threshold

    def _calculate_gain(self, y, y_true, y_false):
        p = len(y_true) / len(y)
        impurity_before = self._gini_impurity(y)
        impurity_after = p * self._gini_impurity(y_true) + (1 - p) * self._gini_impurity(y_false)
        return impurity_before - impurity_after

    def _gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _calculate_leaf_value(self, y):
        classes, counts = np.unique(y, return_counts=True)
        most_common_class = classes[np.argmax(counts)]
        return most_common_class

    def predict(self, X):
        predictions = []

        for arr in X:
            predictions.append(self._next_node(self.root, arr))
        
        return predictions

    def _next_node(self, node, x):
        if node.value is not None:
            return node.value
            
        if x[node.feature] <= node.threshold:
            return self._next_node(node.true_branch, x)
        else:
            return self._next_node(node.false_branch, x)
        
        
def print_tree(node, depth=0):
    indent = "    " * depth
    if node.value is not None:
        print(indent + "Predicted value:", node.value)
    else:
        print(indent + "Feature", node.feature, "<=", node.threshold)
        print(indent + "--> True:")
        print_tree(node.true_branch, depth + 1)
        print(indent + "--> False:")
        print_tree(node.false_branch, depth + 1)

In [88]:
data = load_iris(as_frame=True)
X = data["data"].values
y = data["target"].values

tree = DecisionTree()
tree.fit(X, y)
y_new = tree.predict(X)

print('y_new  y')
for i in range(len(y)):
    print(y_new[i], y[i], sep='      ')

y_new  y
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2

### Часть 2. Случайный лес

##### [task] 1. Усреднение классификаторов (2 балла)
Реализуйте бэггинг над решающими деревьями (усреднение предсказанных вероятностей всего ансамбля или путем "голосования"). В качестве базового алгоритма используйте либо наш класс или DecisionTreeClassifier из пакета scikit-learn. 

##### [task] 2. Бутстрап обучающей выборки (2 балла)
Добавим к нашему усреднению предсказаний бутстрап выборки (генерация случайной выборки того же размера с возвращением). Сгенерируйте с помощью него отдельную обучающую выборку для каждого дерева, обучите их и усредните предсказания, как в предыдущем пункте.

##### [task] 3. Выбор случайного подмножества признаков при построении нового дерева. (2 балла)
Обучайте каждое дерево на отдельной бутстрап-выборке и случайно выбирайте признаки для обучения. 

##### [task] 4. Подсчитайте метрики оценки качества классификации (accuracy, precision, recall, ROC AUC, F-мера) для каждого из вариантов дерева. Сделайте отдельную функцию для подсчета метрик (1 балл)

- Используемый датасет https://archive.ics.uci.edu/dataset/94/spambase
- Все гиперпараметры необходимо вынести аргументы соответсвующей функции.

In [ ]:
# fetch dataset 
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
spambase = fetch_ucirepo(id=94) 
  
# data (as numpy arrays) 
X = spambase.data.features.values
y = spambase.data.targets.values
  
# metadata 
print(spambase.metadata) 
  
# variable information 
print(spambase.variables)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [198]:
#### Your code here

#### Часть 3. 

- [task] Обучите RandomForestClassifier из sklearn на датасете из прошлой части (2 балла)
- [task] Подсчитайте accuracy, precision, recall, ROC AUC, F-мера на отложенной выборке. Получилось лучше или хуже по сравнению с вашим вариантом RandomForest (2 балла)
- Попробуйте объяснить результат.

In [ ]:
#### Your code here